# LDAS: Project - Team 2
Moritz Eck, moritz.eck.0055@student.uu.se<br>
Tyson McLeod, <br>
Isaline Baret, <br>
Markella-Achilleia Zacharouli, <br>

## Setup & Deploy

In [1]:
from pyspark.sql import SparkSession
import matplotlib.pyplot as plt
import time

In [2]:
# start your application with dynamic allocation enabled, a timeout of no more than 30 seconds and a cap on CPU cores:
# REMOTE SESSION
# spark = SparkSession\
#        .builder\
#        .master("spark://192.168.1.153:7077") \
#        .appName("LDSA_Team2_Project")\
#        .config("spark.dynamicAllocation.enabled", True)\
#        .config("spark.shuffle.service.enabled", True)\
#        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
#        .config("spark.executor.cores",4)\
#        .config("spark.files", "./data/heart.csv")\
#        .getOrCreate()

# LOCAL SESSION
spark = SparkSession\
    .builder.master("local[4]")\
    .appName("LDSA_Team2_Project")\
    .config('spark.executor.memory', '6g')\
    .config("spark.driver.memory", "5g")\
    .config("spark.executor.cores",4)\
    .getOrCreate()

# spark context (old RDD)
sc = spark.sparkContext

## Experiment 1: Business Data

### Load and Preprocess Data

In [5]:
# read the CSV as a dataframe (REMOTE/HDFS FILE)
# df = spark_session.read\
#     .option("header", "true")\
#     .csv("hdfs://192.168.1.153:9000/parking-citations.csv")\
#     .cache()

# LOCAL 
# read JSON file
business = spark.read.json("./data/yelp_academic_dataset_business.json").cache()

# the inferred schema can be visualized using the printSchema() method
business.printSchema()

# show top 5 rows
business.show(5)

root
 |-- address: string (nullable = true)
 |-- attributes: struct (nullable = true)
 |    |-- AcceptsInsurance: string (nullable = true)
 |    |-- AgesAllowed: string (nullable = true)
 |    |-- Alcohol: string (nullable = true)
 |    |-- Ambience: string (nullable = true)
 |    |-- BYOB: string (nullable = true)
 |    |-- BYOBCorkage: string (nullable = true)
 |    |-- BestNights: string (nullable = true)
 |    |-- BikeParking: string (nullable = true)
 |    |-- BusinessAcceptsBitcoin: string (nullable = true)
 |    |-- BusinessAcceptsCreditCards: string (nullable = true)
 |    |-- BusinessParking: string (nullable = true)
 |    |-- ByAppointmentOnly: string (nullable = true)
 |    |-- Caters: string (nullable = true)
 |    |-- CoatCheck: string (nullable = true)
 |    |-- Corkage: string (nullable = true)
 |    |-- DietaryRestrictions: string (nullable = true)
 |    |-- DogsAllowed: string (nullable = true)
 |    |-- DriveThru: string (nullable = true)
 |    |-- GoodForDancing: str

In [29]:
# the number of rows
print("Rows in Business Dataframe:\t", business.count())

# the number of RDD partitions
print("Number of Partitions:\t\t", business.rdd.getNumPartitions())

Rows in Business Dataframe:	 192609
Number of Partitions:		 4


In [ ]:
# Creates a temporary view using the DataFrame
# df.createOrReplaceTempView("test")

# SQL statements can be run by using the sql methods provided by spark
# testdf = spark.sql("SELECT name, stars, review_count, hours FROM test WHERE stars BETWEEN 4 AND 5")
# testdf.show()

# SQL statements can be run by using the sql methods provided by spark
# testdf2 = spark.sql("SELECT name, stars, review_count, hours FROM test")
# testdf2.show()

### Experiment 1: Sort all business according to stars and review_count

In [34]:
# top businesses according to starts and review_count
start_time = time.time()
result = business.sort("stars", "review_count", ascending=[0,0])
end_time = time.time()

for row in result.head(20):
    name, stars, rc = row["name"], row["stars"], row["review_count"]
    print("Name:\t{},\tStars:\t{},\tReview Count:\t{}".format(name[:12], stars, rc))
    
print("\nThe evaluation took: {} seconds".format(end_time - start_time))

Name:	Little Miss ,	Stars:	5.0,	Review Count:	1936
Name:	Brew Tea Bar,	Stars:	5.0,	Review Count:	1506
Name:	Eco-Tint,	Stars:	5.0,	Review Count:	679
Name:	Paranormal -,	Stars:	5.0,	Review Count:	662
Name:	Worth Takeaw,	Stars:	5.0,	Review Count:	552
Name:	Carpet Monke,	Stars:	5.0,	Review Count:	552
Name:	Poke Express,	Stars:	5.0,	Review Count:	543
Name:	Fabulous Eye,	Stars:	5.0,	Review Count:	537
Name:	Battlefield ,	Stars:	5.0,	Review Count:	470
Name:	HUMMUS,	Stars:	5.0,	Review Count:	459
Name:	Desert Wind ,	Stars:	5.0,	Review Count:	446
Name:	Free Vegas C,	Stars:	5.0,	Review Count:	433
Name:	Just-In Time,	Stars:	5.0,	Review Count:	422
Name:	Legacy Air,	Stars:	5.0,	Review Count:	416
Name:	Kodo Sushi S,	Stars:	5.0,	Review Count:	414
Name:	Studio 21 Ta,	Stars:	5.0,	Review Count:	412
Name:	Not Your Typ,	Stars:	5.0,	Review Count:	402
Name:	FINO for MEN,	Stars:	5.0,	Review Count:	387
Name:	Novel Ice Cr,	Stars:	5.0,	Review Count:	386
Name:	Zenaida's Ca,	Stars:	5.0,	Review Count:	374

The evalu

## Experiment 2: User Data

### Load User Data & Preprocess

In [ ]:
# LOCAL 
# read JSON file
users = spark.read.json("./data/yelp_academic_dataset_user.json")

# the inferred schema can be visualized using the printSchema() method
users.printSchema()

# the number of rows
print("Rows in Users Dataframe:\t", users.count())

# the number of RDD partitions
print("Number of Partitions:\t\t", users.rdd.getNumPartitions())

In [3]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType, IntegerType

# use udf to define a row-at-a-time udf
def count_friends(line):
    # lowercase transformation
    # splitting into tokens/words
    return len(line.lower().split(', '))

# count the number of friends per user and add the value as a new column
count_friends = udf(count_friends, IntegerType())
users = users.withColumn("friendsCount", count_friends(col("friends")))

# print first three rows
print(users.show(3))

root
 |-- average_stars: double (nullable = true)
 |-- compliment_cool: long (nullable = true)
 |-- compliment_cute: long (nullable = true)
 |-- compliment_funny: long (nullable = true)
 |-- compliment_hot: long (nullable = true)
 |-- compliment_list: long (nullable = true)
 |-- compliment_more: long (nullable = true)
 |-- compliment_note: long (nullable = true)
 |-- compliment_photos: long (nullable = true)
 |-- compliment_plain: long (nullable = true)
 |-- compliment_profile: long (nullable = true)
 |-- compliment_writer: long (nullable = true)
 |-- cool: long (nullable = true)
 |-- elite: string (nullable = true)
 |-- fans: long (nullable = true)
 |-- friends: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- name: string (nullable = true)
 |-- review_count: long (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)
 |-- yelping_since: string (nullable = true)

+-------------+---------------+---------------+----------------+-----

### Experiment 2: Sorting dataset according to different fields

In [6]:
start_time = time.time()

# # top reviewers
top_reviewers = users.sort("review_count", ascending=False).head(20)

# # top useful reviews
top_useful_reviews = users.sort("useful", "review_count", ascending=[0,0]).head(20)

# top # of fans per reviewer
top_fan_count = users.sort("fans", "useful", ascending=[0,0]).head(20)

# top # of friends per reviewer
top_friends = users.sort("friendsCount", "fans", ascending=[0,0]).head(20)

end_time = time.time()

print("Top 5 Reviewers by Review Count!")
for row in top_reviewers[:5]:
    name, since, rc = row["name"], row["yelping_since"], row["review_count"]
    print("Name:\t{}\tReview Count:\t{}\tYelping Since:\t{}".format(name, rc, since))
    
print("\nTop 5 Most Useful Reviews by Reviewer!")
for row in top_useful_reviews[:5]:
    name, since, rc, useful = row["name"], row["yelping_since"], row["review_count"], row["useful"]
    print("Name:\t{}\tUseful Reviews:\t{}\tReview Count:\t{}\tYelping Since:\t{}".format(name, useful, rc, since))
    
print("\nTop 5 Reviewers with most Fans!")
for row in top_fan_count[:5]:
    name, since, rc, useful, fans = row["name"], row["yelping_since"], row["review_count"], row["useful"], row["fans"]
    print("Name:\t{}\tFans:\t{}\tUseful Reviews:\t{}\tReview Count:\t{}\tYelping Since:\t{}".format(name, fans, useful, rc, since))
    
print("\nTop 5 Reviewers with most Friends!")
for row in top_fan_count[:5]:
    name, since, fc, fans = row["name"], row["yelping_since"], row["friendsCount"], row["fans"]
    print("Name:\t{}\tFriends:\t{}\tFans:\t{}\tYelping Since:\t{}".format(name, fans, fc, since))
    
print("\nThe evaluation took: {:3.3f} seconds".format(end_time - start_time))

Top 5 Reviewers by Review Count!
Name:	Victor	Review Count:	13278	Yelping Since:	2007-12-08 14:56:45
Name:	Shila	Review Count:	12390	Yelping Since:	2010-10-17 06:35:06
Name:	Bruce	Review Count:	10022	Yelping Since:	2009-03-08 21:47:44
Name:	Kim	Review Count:	9821	Yelping Since:	2006-05-31 21:27:42
Name:	George	Review Count:	7750	Yelping Since:	2009-11-06 22:53:16

Top 5 Most Useful Reviews by Reviewer!
Name:	Harald	Useful Reviews:	154202	Review Count:	1350	Yelping Since:	2012-11-27 14:19:33
Name:	Richard	Useful Reviews:	99162	Review Count:	1086	Yelping Since:	2009-08-23 13:03:27
Name:	Maggie	Useful Reviews:	89792	Review Count:	2830	Yelping Since:	2008-11-30 02:47:32
Name:	Fox	Useful Reviews:	89418	Review Count:	6407	Yelping Since:	2009-05-26 11:33:58
Name:	Rohlin	Useful Reviews:	81003	Review Count:	875	Yelping Since:	2010-01-11 01:50:32

Top 5 Reviewers with most Fans!
Name:	Mike	Fans:	9538	Useful Reviews:	19715	Review Count:	1634	Yelping Since:	2009-04-25 14:54:26
Name:	Katie	Fans:	29

## Experiment 3: Reviews

### Load User Data & Preprocess

In [19]:
# LOCAL 
# read JSON file
reviews = spark.read.json("./data/reviews_1000000.json")

# the inferred schema can be visualized using the printSchema() method
reviews.printSchema()

# the number of rows
print("Rows in Reviews Dataframe:\t", reviews.count())

# the number of RDD partitions
print("Number of Partitions:\t\t", reviews.rdd.getNumPartitions())

root
 |-- business_id: string (nullable = true)
 |-- cool: long (nullable = true)
 |-- date: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- review_id: string (nullable = true)
 |-- stars: double (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)

Rows in Reviews Dataframe:	 1000001
Number of Partitions:		 6


In [24]:
start_time = time.time()

from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType

# use udf to define a row-at-a-time udf
def preprocess(line):
    # lowercase transformation
    # splitting into tokens/words
    tokens = line.lower().split(' ')
    tokens = [token.strip() for token in tokens]
    return str(tokens)

# count the number of friends per user and add the value as a new column
tok = udf(preprocess, StringType())
pr_reviews = reviews.withColumn("text", tok(col("text")))

end_time = time.time()
    
print("\nThe evaluation took: {:3.3f} seconds".format(end_time - start_time))

# print first three rows
pr_reviews.show(5, False)


The evaluation took: 0.024 seconds
+----------------------+----+-------------------+-----+----------------------+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

### Experiment 3: 

## Shutdown Session

In [25]:
# release the cores for another application!
spark.stop()
sc.stop()